In [31]:
import obspy
import os, glob, sys
import numpy as np
import pandas as pd
from obspy.signal.filter import bandpass
import matplotlib.pyplot as plt
from scipy.fftpack import fft, fftfreq
from matplotlib.ticker import LogFormatter

import matplotlib.cm as cm
import matplotlib.colors as mcolors
import matplotlib.dates as mdates
import matplotlib.patches as patches

In [3]:
comp = ['ZN', 'ZE', 'NE']
sta_list = np.loadtxt('stalst.redb', dtype='str')
sta_list = sta_list[1:] 
# sta_list = np.loadtxt('stalst.fix', dtype='str')
print(sta_list)


['001' '002' '003' '004' '005' '006a' '006b']


In [4]:
dalst=[]
with open("evtlst.redb", "r") as file:
    for line in file:
        fields = line.split()
        # if fields and int(fields[0]) > 20230601 and int(fields[0]) < 20231110 and (fields[0][7] == "1" or fields[0][7] == "3" or fields[0][7] == "5" or fields[0][7] == "8") :
        if fields and int(fields[0]) > 20230601 :
            dalst.append(fields[0])
print(dalst)

['20230602', '20230603', '20230604', '20230605', '20230606', '20230607', '20230608', '20230609', '20230610', '20230611', '20230612', '20230613', '20230614', '20230615', '20230616', '20230617', '20230618', '20230619', '20230620', '20230621', '20230622', '20230623', '20230624', '20230625', '20230626', '20230627', '20230628', '20230629', '20230630', '20230701', '20230702', '20230703', '20230704', '20230705', '20230706', '20230707', '20230708', '20230709', '20230710', '20230711', '20230712', '20230713', '20230714', '20230715', '20230716', '20230717', '20230718', '20230719', '20230720', '20230721', '20230722', '20230723', '20230724', '20230725', '20230726', '20230727', '20230728', '20230729', '20230730', '20230731', '20230801', '20230802', '20230803', '20230804', '20230805', '20230806', '20230807', '20230808', '20230809', '20230810', '20230811', '20230812', '20230813', '20230814', '20230815', '20230816', '20230817', '20230818', '20230819', '20230820', '20230821', '20230822', '20230823', '20

In [5]:
def plot_WaveSpec(dates, all_waveforms, lagtime, all_spectra, half_freqs, st, comp, target, outfd, suffix):
  # Convert valid_dalst to datetime objects
  dates = pd.to_datetime(valid_dalst, format='%Y%m%d')
  # Determine the min and max values for the color bar

  # Plotting
  fig, ax = plt.subplots(1, 2, figsize=(24, 6), )# gridspec_kw={'width_ratios': [2, 1]})
  fig.suptitle(f"Waveform and Spectrum for {st} - {comp} Component  {target}, {suffix} ", fontsize=16)

  # Plot the waveform using imshow
  c1 = ax[0].imshow(all_waveforms.T, aspect='auto', cmap="RdBu", 
                    extent=[0, len(dates),  lagtime[mask].max(),  lagtime[mask].min()], 
                    vmin=-0.1, vmax=0.1)
                  #   vmin=-max(abs(vmin), abs(vmax))*0.5, vmax=max(abs(vmin), abs(vmax))*0.5)
  fig.colorbar(c1, ax=ax[0], label="Amplitude")


  ax[0].set_title(f"SCFs over time for {st} - {comp} Component")
  ax[0].set_ylabel("Lag Time (s)")

  ax[1].set_title(f"Amplitude Spectrum over time for {st} - {comp} Component")
  ax[1].set_ylabel("Frequency (Hz)")


  # Plot the waveform using imshow
  c2 = ax[1].imshow(all_spectra.T, aspect='auto', cmap="viridis", 
                    extent=[0, len(dates), half_freqs.max(),  half_freqs.min()],
                    # extent=[0, len(dates), 9 , 0],
                    norm=mcolors.LogNorm(vmin=1, vmax=5)) 
  fig.colorbar(c2, ax=ax[1], label="Amplitude " , ticks=[ 0.1,1, 10])

  for pk in range(2):
    ax[pk].set_xticks(np.arange(0, len(dates), 10))  # Set xticks every 10 days
    ax[pk].set_xticklabels(dates[::10].strftime('%Y-%m-%d'), )
    ax[pk].tick_params(axis="x", rotation=90)  # Rotate labels for better readability
    ax[pk].set_xlabel("Date")
    ax[pk].grid(True)
    
  plt.tight_layout()
  plt.savefig(f'{outfd}/Spec_{st}_{cmp}_{suffix}_{target}.png', dpi=200)
  plt.close()

In [58]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import pandas as pd

def plot_WaveSpec_all(valid_dalst, all_waveforms, lagtime, all_spectra, half_freqs, st, comp, target, outfd, suffix, coda):
    # Step 1: Convert date strings to datetime
    dates = pd.to_datetime(valid_dalst, format='%Y%m%d')

    # Step 2: Create a full date range from min to max
    full_dates = pd.date_range(start=dates.min(), end=dates.max(), freq='D')

    # Step 3: Build empty arrays for all_waveforms and all_spectra
    n_full_days = len(full_dates)
    n_lags = all_waveforms.shape[1]
    n_freqs = all_spectra.shape[1]

    full_waveforms = np.full((n_full_days, n_lags), np.nan)
    full_spectra = np.full((n_full_days, n_freqs), np.nan)

    # Step 4: Map existing data into full arrays
    date_idx_map = {d: i for i, d in enumerate(full_dates)}
    for i, d in enumerate(dates):
        idx = date_idx_map.get(d)
        if idx is not None:
            full_waveforms[idx, :] = all_waveforms[i, :]
            full_spectra[idx, :] = all_spectra[i, :]

    # Step 5: Plotting
    fig, ax = plt.subplots(1, 2, figsize=(24, 6))
    fig.suptitle(f"Waveform and Spectrum for {st} - {comp} Component  {target}, {suffix}", fontsize=22)
    
    for pk in range(2):
        ax[pk].set_xticks(np.arange(0, n_full_days, 10))
        ax[pk].set_xticklabels(full_dates[::10].strftime('%Y-%m-%d'), rotation=90, fontsize=16)
        ax[pk].set_xlabel("Date", fontsize=20)
        ax[pk].grid(True)
    
    c1 = ax[0].imshow(np.ma.masked_invalid(full_waveforms).T, aspect='auto', cmap="RdBu",
                      extent=[0, n_full_days, lagtime.max(), lagtime.min()],
                      vmin=-0.1, vmax=0.1)
    fig.colorbar(c1, ax=ax[0], ).ax.tick_params(labelsize=16)
    ax[0].set_title(f"SCFs over time for {st} - {comp} Component", fontsize=22)
    ax[0].set_ylabel("Lag Time (s)", fontsize=20)
    
    # Add box from lag time 1–3 seconds across all dates
    rect = patches.Rectangle(
        (0, coda[1]),                      # (x, y) = (start date index, lower lag time)
        n_full_days,                # width spans all dates
        1 - 3,                      # height (Note: y grows downward with imshow)
        linewidth=2,
        edgecolor='black',
        facecolor='none',
        linestyle='--'
    )
    ax[0].add_patch(rect)

    c2 = ax[1].imshow(np.ma.masked_invalid(full_spectra).T, aspect='auto', cmap="viridis",
                      extent=[0, n_full_days, half_freqs.max(), half_freqs.min()],
                      norm=mcolors.LogNorm(vmin=1, vmax=5))
    formatter = LogFormatter(labelOnlyBase=False)
    fig.colorbar(c2, ax=ax[1], ticks=[1,2,3,4,5], format=formatter).ax.tick_params(labelsize=16)
    ax[1].set_title(f"Amplitude Spectrum over time for {st} - {comp} Component", fontsize=22)
    ax[1].set_ylabel("Frequency (Hz)", fontsize=20)
    ax[1].tick_params(labelsize=18)
    ax[0].tick_params(labelsize=18)

    plt.tight_layout()
    # plt.show()
    plt.savefig(f'{outfd}/Specall_{st}_{comp}_{suffix}_{target}.png', dpi=200)
    plt.close()


In [7]:
# sta_list=['005']
listdir = sorted([f for f in glob.glob("COR_sn5_20Hz_F?-?")])
# listdir=['COR_sn5_20Hz_F4-8']
# listdir=['COR_sn5_20Hz_F2-4']

In [60]:
outfd='Check_WaveSpectra'
suffix='ave12-0'
# target='COR_sn5_20Hz_F2-4'

for ndir, target in enumerate(listdir):
    print(target, suffix)
    if ndir ==0:
        coda=[1, 3]  # Define the coda window for the first directory
    else:
        coda=[0.5,2]
    for st in sta_list:
        for cmp in comp:
            # Storage for amplitude values
            all_amplitudes = []
            all_spectra = []
            valid_dalst = []
            all_waveforms = []
            for da in dalst:
                # print(f"Processing: {da}")

                fd = f'{target}/{st}-{st}'
                # pattern = os.path.join(fd, f'COR_{st}_{st}_{da}.SAC_{cmp}')
                pattern = os.path.join(fd, f'COR_{st}_{st}_{da}.SAC_{cmp}_{suffix}_s')  # Select ZN component
                sacfs = sorted(glob.glob(pattern))

                if not sacfs:
                    # # print(f"Warning: No file found for {da} {target}")
                    continue
                
                valid_dalst.append(da)  # Keep the date if there are corresponding SAC files

                sac = obspy.read(sacfs[0])  # Read SAC file
                ccf = sac[0].data           # Extract cross-correlation function

                sampling_rate = sac[0].stats.sampling_rate
                dt = sac[0].stats.delta
                npts = sac[0].stats.npts
                # print(sacf, 'sampling_rate',sampling_rate, dt, npts)
                
                if npts < 20000:
                    lagtime = np.arange(-100+dt, 100, dt)
                if npts > 20000:
                    lagtime = np.arange(-100-dt, 100, dt)
                if npts == 20000:
                    lagtime = np.arange(-100, 100, dt) 

                # Select the waveform segment between 0-3s
                mask = (lagtime >= 0) & (lagtime <= 6)
                ccf_cut = ccf[mask]
                lag_cut = lagtime[mask]

                all_waveforms.append(ccf_cut)
                    

            # Loop over each waveform to compute its spectrum
            for waveform in all_waveforms:
                # Compute the amplitude of each individual waveform
                N = len(waveform)
                freqs = fftfreq(N, d=dt)  # Frequency axis based on sampling interval dt
                spectrum = np.abs(fft(waveform))  # FFT Amplitude Spectrum

                # Keep only the positive frequencies (first half of the spectrum)
                all_spectra.append(spectrum[:N//2])

                # Store the waveform's amplitude (not averaged)
                all_amplitudes.append(waveform)
                
            # Convert lists to numpy arrays for plotting
            all_waveforms = np.array(all_waveforms)  # Shape: (len(dalst), len(lagtime_cut))
            all_amplitudes = np.array(all_amplitudes)  # Shape: (len(dalst), len(lagtime_cut))
            all_spectra = np.array(all_spectra)  # Shape: (len(dalst), len(freqs//2))
            half_freqs = freqs[:N//2]
            # Convert date strings to numerical indices
            date_indices = np.arange(len(valid_dalst))
            plot_WaveSpec_all(valid_dalst, all_waveforms, lag_cut, all_spectra, half_freqs, st, cmp, target, outfd, suffix, coda)


        

COR_sn5_20Hz_F2-4 ave12-0
COR_sn5_20Hz_F4-8 ave12-0


In [10]:
stop

NameError: name 'stop' is not defined

In [ ]:
da='20230821'
# CORFD='COR_sn5_20Hz_F2-4'
outfd=f'./Check_COR_para_dalst'
suffix='ave12-0'
amp=[1,1,2,4]
listdir = ([f for f in glob.glob("COR_sn5_*")])

for da in dalst:
    print(da)
    for st in sta_list:
        
        fig, ax= plt.subplots(3,6, figsize=(24, 10) )
        for i, target in enumerate(listdir):
            # target='COR_sn0_20Hz_F2-4'
            #print(st, target)

            for fk, fband in enumerate(['unfiltered','2-4', '2-4', '2-4']):
                if fk>0:
                    fmin, fmax = fband.split('-')

                for k, cmp in enumerate(comp):
                    fd = f'{target}/{st}-{st}'
                    pattern = os.path.join(fd, f'COR_{st}_{st}_{da}.SAC_{cmp}_{suffix}_s')  # Use glob pattern
                    sacfs = sorted(glob.glob(pattern))  # Use glob instead of os.listdir

                    if not sacfs:  # Debugging check
                        print(f"Warning: No files found matching {pattern}")
                        break 
                    
                    sacf = sacfs[0]
                    sac = obspy.read(sacf)
                    ccf = sac[0].data
                    sampling_rate = sac[0].stats.sampling_rate
                    dt = sac[0].stats.delta
                    npts = sac[0].stats.npts
                    # print(sacf, 'sampling_rate',sampling_rate, dt, npts)
                    
                    if npts < 20000:
                        lagtime = np.arange(-100+dt, 100, dt)
                    if npts > 20000:
                        lagtime = np.arange(-100-dt, 100, dt)
                    if npts == 20000:
                        lagtime = np.arange(-100, 100, dt)
            
                    # Save unfiltered trace for spectrogram
                    if fk == 0:
                        ccf_unfiltered = ccf.copy()
                    
                    if fk > 0:
                        ccf = bandpass(ccf, float(fmin), float(fmax), sampling_rate, corners=4, zerophase=True)
                    
                    # Convert YYYYMMDD to YYYY/MM/DD format
                    formatted_date = f"{da[:4]}-{da[4:6]}-{da[6:]}" 

                    ax[k,fk].plot(lagtime, ccf /np.max(np.abs(ccf)) * amp[fk] , linewidth=1, label=f"{target}") 
    
                    ax[k,0].set_xlim(-4,4)
                    ax[k,1].set_xlim(-2,2)
                    ax[k,2].set_xlim(0,3)
                    ax[k,3].set_xlim(1,3)
                    
                    if fk > 0:
                        ax[k,fk].set_title(f' {st}, {cmp} @ {fmin}-{fmax} Hz {suffix} on {da} \n Amp x {amp[fk]}')
                    else:
                        ax[k,fk].set_title(f'COR versions on {formatted_date}\n Station {st}     {cmp}  Amp x {amp[fk]}')
                    
                    ax[k,fk].grid('True')
                    ax[k,fk].set_xlabel("Lag Time (s)")
                    ax[k,fk].set_ylim(-1,1)
                    
                    
                    
                    # Cut the waveform between lagtime 0 and 3 seconds
                    mask = (lagtime >= 1) & (lagtime <= 3)
                    lagtime_cut = lagtime[mask]
                    ccf_cut = ccf[mask]
                    
                    if fk == 3:
                        # Plot the frequency spectrum for the unfiltered trace in the new extra column (column index 3)
                        n = len(ccf_cut)
                        freqs = np.fft.rfftfreq(n, d=dt)
                        spectrum = np.abs(np.fft.rfft(ccf_cut))
                        ax[k, 4].plot(freqs, spectrum, linewidth=1, alpha=0.75, label=f"{target}")
                        ax[k, 4].set_title(f'Frequency Spectrum {st} {cmp}\n cut 1-3 sec on {formatted_date}')
                        ax[k, 4].set_xlabel('Frequency (Hz)')
                        ax[k, 4].set_ylabel('Amplitude')
                        ax[k, 4].set_xlim(0.1,6.2)
                        ax[k, 4].set_yscale('log')
                        ax[k, 4].grid(True)
                        
                        # Plot the phase spectrum for the unfiltered trace in the new extra column (column index 5)
                        phase = np.angle(np.fft.rfft(ccf_cut))
                        ax[k, 5].plot(freqs, phase, linewidth=1, alpha=0.75, )
                        ax[k, 5].set_title(f'Phase Spectrum {st} {cmp}\n cut 1-3 sec on {formatted_date}')
                        ax[k, 5].set_xlabel('Frequency (Hz)')
                        ax[k, 5].set_ylabel('Phase (radians)')
                        ax[k, 5].set_xlim(0.1,6.2)
                        ax[k, 5].set_ylim(-np.pi, np.pi)
                        ax[k, 5].grid(True)
                    
            ax[k,0].set_ylabel("Rescaled Amplitude")
        ax[0,0].legend(loc='upper right', )
        plt.tight_layout()
        # plt.show()
        plt.savefig(f'{outfd}/Waveform_{st}_{suffix}_{da}.png', dpi=200)
        plt.close()
        # stop


In [ ]:
stop

In [ ]:
da='20230821'
# CORFD='COR_sn5_20Hz_F2-4'
outfd=f'./Check_COR_para_dalst'
suffix='ave12-0'
amp=[1,1,2,4]
listdir = ([f for f in glob.glob("COR_sn5_*")])

for da in dalst:
    print(da)
    for st in sta_list:
        
        fig, ax= plt.subplots(3,4, figsize=(16, 10) )
        for i, target in enumerate(listdir):
            # target='COR_sn0_20Hz_F2-4'
            #print(st, target)

            for fk, fband in enumerate(['unfiltered','2-4', '2-4', '2-4']):
                if fk>0:
                    fmin, fmax = fband.split('-')

                for k, cmp in enumerate(comp):
                    fd = f'{target}/{st}-{st}'
                    pattern = os.path.join(fd, f'COR_{st}_{st}_{da}.SAC_{cmp}_{suffix}_s')  # Use glob pattern
                    sacfs = sorted(glob.glob(pattern))  # Use glob instead of os.listdir

                    if not sacfs:  # Debugging check
                        print(f"Warning: No files found matching {pattern}")
                        break 
                    
                    sacf = sacfs[0]
                    sac = obspy.read(sacf)
                    ccf = sac[0].data
                    sampling_rate = sac[0].stats.sampling_rate
                    dt = sac[0].stats.delta
                    npts = sac[0].stats.npts
                    # print(sacf, 'sampling_rate',sampling_rate, dt, npts)
                    
                    if npts < 20000:
                        lagtime = np.arange(-100+dt, 100, dt)
                    if npts > 20000:
                        lagtime = np.arange(-100-dt, 100, dt)
                    if npts == 20000:
                        lagtime = np.arange(-100, 100, dt)
                    
                    if fk > 0:
                        ccf = bandpass(ccf, float(fmin), float(fmax), sampling_rate, corners=4, zerophase=True)
                    
                    # Convert YYYYMMDD to YYYY/MM/DD format
                    formatted_date = f"{da[:4]}-{da[4:6]}-{da[6:]}" 

                    ax[k,fk].plot(lagtime, ccf /np.max(np.abs(ccf)) * amp[fk] , linewidth=1, label=f"{target}") 
    
                    ax[k,0].set_xlim(-4,4)
                    ax[k,1].set_xlim(-2,2)
                    ax[k,2].set_xlim(0,2)
                    ax[k,3].set_xlim(1,3)
                    
                    if fk > 0:
                        ax[k,fk].set_title(f' {st}, {cmp} @ {fmin}-{fmax} Hz {suffix} on {da} \n Amp x {amp[fk]}')
                    else:
                        ax[k,fk].set_title(f'COR versions on {formatted_date}\n Station {st}     {cmp}  Amp x {amp[fk]}')
                    
                    ax[k,fk].grid('True')
                    ax[k,fk].set_xlabel("Lag Time (s)")
                    ax[k,fk].set_ylim(-1,1)            
            ax[k,0].set_ylabel("Rescaled Amplitude")
        ax[0,0].legend(loc='upper right', )
        plt.tight_layout()
        # plt.show()
        plt.savefig(f'{outfd}/Waveform_{st}_{suffix}_{da}.png')
        plt.close()
        # stop


In [ ]:
stop

In [ ]:

# CORFD='COR_sn5_20Hz_F2-4'
outfd=f'./Check_COR_para'
suffix='ave12-0'
amp=[10,10,20,80]

# Read swap dates from file
swap_ranges = []
with open("swap.redb", "r") as f:
    for line in f:
        start, end = line.split()
        swap_ranges.append((int(start)+20000000, int(end)+20000000))  # Convert to int for comparison

color_sets = ["red", "blue", "green", "purple", "orange"]

for i, target in enumerate(listdir):
    # target='COR_sn0_20Hz_F2-4'
    # print(target)
    for st in sta_list:
        fig, ax= plt.subplots(3,4, figsize=(14, 16), sharey=True )
        for fk, fband in enumerate(['unfiltered','2-4', '2-4', '2-4']):
            if fk>0:
                fmin, fmax = fband.split('-')
                
            for k, cmp in enumerate(comp):
                fd = f'{target}/{st}-{st}'
                pattern = os.path.join(fd, f'COR_{st}_{st}_????????.SAC_{cmp}_{suffix}_s')  # Use glob pattern
                # pattern = os.path.join(fd, f'COR_{st}_{st}_????????.SAC_{cmp}')  # Use glob pattern
                sacfs = sorted(glob.glob(pattern))  # Use glob instead of os.listdir
                
                ax[k,0].set_xlim(-4,4)
                ax[k,1].set_xlim(-2,2)
                ax[k,2].set_xlim(0,2)
                ax[k,3].set_xlim(1,3)
                
                if fk > 0:
                    ax[k,fk].set_title(f' {st} :  {cmp} @ {fmin}-{fmax} Hz, {suffix}')
                    # ax[k,fk].set_title(f' {st} :  {cmp} @ {fmin}-{fmax} Hz')
                else:
                    ax[k,fk].set_title(f'{st} {cmp} {target}')
                
                if not sacfs:  # Debugging check
                    print(f"Warning: No files found matching {pattern}")
                waveforms = []
                dates = []
                for nsac, sacf in enumerate(sacfs):
                    datename=sacf.split('/')[-1].split('_')[3].split('.')[0]
                    dates.append(datename)
                    
                    sac = obspy.read(sacf)
                    ccf = sac[0].data
                    sampling_rate = sac[0].stats.sampling_rate
                    dt = sac[0].stats.delta
                    npts = sac[0].stats.npts
                    if npts < 20000:
                        lagtime = np.arange(-100+dt, 100, dt)
                    if npts > 20000:
                        lagtime = np.arange(-100-dt, 100, dt)
                    if npts == 20000:
                        lagtime = np.arange(-100, 100, dt)
                    
                    # lagtime = np.arange(-100, 100-dt, dt)
                    if fk > 0:
                        ccf = bandpass(ccf, float(fmin), float(fmax), sampling_rate, corners=4, zerophase=True)
                    waveforms.append(ccf)
                    
                # Convert dates to numerical values for the y-axis
                date_nums = np.arange(len(dates))
                dates = [str(date) for date in dates]
                # Convert YYYYMMDD to YYYY/MM/DD format
                formatted_dates = [f"{d[:4]}-{d[4:6]}-{d[6:]}" for d in dates]

                
                # Plot each waveform with an offset
                for i, (date, waveform) in enumerate(zip(date_nums, waveforms)):
                    # for swap_idx, swap_range in enumerate(swap_ranges):
                    #     if (swap_range[0] <= int(dates[i])) and (int(dates[i]) <= swap_range[1]):
                    #         color = color_sets[swap_idx]  # Assign corresponding color
                    #         break
                    # print( i, swap_idx, color)
                    ax[k,fk].plot(lagtime, waveform * amp[fk] + i , linewidth=0.75,)  
                # Select every n th date for labeling
                tick_indices = np.arange(0, len(dates), 6)
                ax[k,fk].set_yticks(tick_indices)
                ax[k,fk].set_yticklabels([formatted_dates[i] for i in tick_indices])
                ax[k,fk].set_xlabel("Lag Time (s)")
                
                ax[k,fk].grid('True')
                ax[k,fk].set_ylim(-5, len(dates)+5)
        ax[k,0].set_ylabel("Date")
        plt.tight_layout()
        # plt.show()
        plt.savefig(f'{outfd}/Check_{st}_{suffix}_{target}.png')

    # stop
